# 서포트 벡터 머신(Support Vector Machines)

- 회귀, 분류, 이상치 탐지 등에 사용되는 지도학습 방법
- 클래스 사이의 경계에 위치한 데이터 포인트를 서포트 벡터(support vector)라고 한다.
- 각 지지 벡터가 클래스 사이의 결정 경계를 구분하는데 얼마나 중요한지를 학습
- 각 지지 벡터 사이의 마진이 가장 큰 방향으로 학습
- 지지 벡터 까지의 거리와 지지 벡터의 중요도를 기반으로 예측을 수행

![support vector machine](https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Svm_separating_hyperplanes.png/220px-Svm_separating_hyperplanes.png)

- H3은 두 클래스의 점들을 제대로 분류하고 있지 않음
- H1과 H2는 두 클래스의 점들을 분류하는데, H2가 H1보다 더 큰 마진을 갖고 분류하는 것을 확인할 수 있음

In [1]:
import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(['seaborn-whitegrid'])

/var/folders/c2/h79928n92258kzkwtzbxjbj80000gn/T/ipykernel_61588/265178491.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-whitegrid'])


In [3]:
from sklearn.svm import SVR, SVC 
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer, load_iris, load_wine
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE

In [7]:
#from sklearn.datasets import load_boston
import pickle

!wget https://raw.githubusercontent.com/myoh0623/dataset/main/boston.pickle

with open('./boston.pickle', 'rb') as f:
    boston = pickle.load(f)

print(boston.keys())
print(boston.DESCR)

zsh:1: command not found: wget
dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])
.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of 

## SVM을 이용한 회귀 모델과 분류 모델

### SVM을 사용한 회귀 모델 (SVR)

In [10]:
x = boston.data
y = boston.target
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=123)

model = SVR()
model.fit(x_train, y_train)

print(f'Train Data Score: {model.score(x_train, y_train)}')
print(f'Test Data Score: {model.score(x_test, y_test)}')

Train Data Score: 0.2177283706374875
Test Data Score: 0.13544178468518187


### SVM을 사용한 분류 모델 (SVC)

In [11]:
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=123)

model = SVC()
model.fit(x_train, y_train)

print(f'Train Data Score: {model.score(x_train, y_train)}')
print(f'Test Data Score: {model.score(x_test, y_test)}')

Train Data Score: 0.9014084507042254
Test Data Score: 0.9230769230769231


## 커널 기법

- 입력 데이터를 고차원 공간에 사상해서 비선형 특징을 학습할 수 있도록 확장하는 방법
- scikit-learn에서는 Linear, Polynomial, RBF(Radial Basis Function)등 다양한 커널 기법을 지원

In [12]:
# load_boston 보스턴 집값

In [13]:
x = boston.data
y = boston.target
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=123)

linear_svr = SVR(kernel = 'linear')
linear_svr.fit(x_train, y_train)

print(f'Linear SVR Train Data Score: {linear_svr.score(x_train, y_train)}')
print(f'Linear SVR Test Data Score: {linear_svr.score(x_test, y_test)}')

polynomial_svr = SVR(kernel = 'poly')
polynomial_svr.fit(x_train, y_train)

print(f'Polynomial SVR  Train Data Score: {polynomial_svr.score(x_train, y_train)}')
print(f'Polynomial SVR Test Data Score: {polynomial_svr.score(x_test, y_test)}')

rbf_svr = SVR(kernel = 'rbf')
rbf_svr.fit(x_train, y_train)

print(f'RBF SVR Train Data Score: {rbf_svr.score(x_train, y_train)}')
print(f'RBF SVR Test Data Score: {rbf_svr.score(x_test, y_test)}')

Linear SVR Train Data Score: 0.7155065522120606
Linear SVR Test Data Score: 0.638039631835579
Polynomial SVR  Train Data Score: 0.2024454261446288
Polynomial SVR Test Data Score: 0.13366845036746255
RBF SVR Train Data Score: 0.2177283706374875
RBF SVR Test Data Score: 0.13544178468518187


In [14]:
# 유방암 데이터를 이용한 분류
# - 각 커널별로 유방암 데이터를 분류해보자. 

x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=123)

linear_svr = SVR(kernel = 'linear')
linear_svr.fit(x_train, y_train)

print(f'Linear SVR Train Data Score: {linear_svr.score(x_train, y_train)}')
print(f'Linear SVR Test Data Score: {linear_svr.score(x_test, y_test)}')

polynomial_svr = SVR(kernel = 'poly')
polynomial_svr.fit(x_train, y_train)

print(f'Polynomial SVR  Train Data Score: {polynomial_svr.score(x_train, y_train)}')
print(f'Polynomial SVR Test Data Score: {polynomial_svr.score(x_test, y_test)}')

rbf_svr = SVR(kernel = 'rbf')
rbf_svr.fit(x_train, y_train)

print(f'RBF SVR Train Data Score: {rbf_svr.score(x_train, y_train)}')
print(f'RBF SVR Test Data Score: {rbf_svr.score(x_test, y_test)}')


Linear SVR Train Data Score: 0.29804873795549414
Linear SVR Test Data Score: 0.13907362733287165
Polynomial SVR  Train Data Score: 0.5492121168675737
Polynomial SVR Test Data Score: 0.33076721394141473
RBF SVR Train Data Score: 0.7141833202421759
RBF SVR Test Data Score: 0.7625209396028005


## 매개변수 튜닝

- SVM은 사용하는 커널에 따라 다양한 매개변수 설정 가능
- 매개변수를 변경하면서 성능변화를 관찰

 - C는 얼마나 많은 데이터 샘플이 다른 클래스에 놓이는 것을 허용하는지를 결정
  - 작을 수록 많이 허용하고, 클 수록 적게 허용한다.
  -  C값을 낮게 설정하면 이상치들이 있을 가능성을 크게 잡아 일반적인 결정 경계를 찾아내고, 높게 설정하면 반대로 이상치의 존재 가능성을 작게 봐서 좀 더 세심하게 결정 경계를 찾아낸다.

  - [참고](https://bskyvision.com/entry/%EC%84%9C%ED%8F%AC%ED%8A%B8-%EB%B2%A1%ED%84%B0-%EB%A8%B8%EC%8B%A0SVM%EC%9D%98-%EC%82%AC%EC%9A%A9%EC%9E%90%EB%A1%9C%EC%84%9C-%EA%BC%AD-%EC%95%8C%EC%95%84%EC%95%BC%ED%95%A0-%EA%B2%83%EB%93%A4-%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98-C%EC%99%80-gamma)


|파라미터|default	| 설명|
|-------|-------|-------|
|C	|1.0|	오류를 얼마나 허용할 것인지 (규제항) 클수록 하드마진, 작을수록 소프트마진에 가까움|
|kernel	|'rbf' (가우시안 커널)| 'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'|
|degree	| 3| 	다항식 커널의 차수 결정 |
|gamma	|'scale'|	결정경계를 얼마나 유연하게 그릴지 결정 클수록 오버피팅 발생 가능성 높아짐|
|coef0	|0.0	|다항식 커널에 있는 상수항 r

In [15]:
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=123)

In [ ]:
polynomial_svc = SVC(kernel = 'poly', degree=2, C=0.1, gamma='auto')
polynomial_svc.fit(x_train, y_train)

print(f'kernel=ploy, degree={2}, C={0.1}, gamma={"auto"}')
print(f'Polynomial SVC Train Data Score: {polynomial_svc.score(x_train, y_train)}')
print(f'Polynomial SVC Test Data Score: {polynomial_svc.score(x_test, y_test)}')

In [10]:
rbf_svc = SVC(kernel = 'rbf', C=2.0, gamma='scale')
rbf_svc.fit(x_train, y_train)

print(f'kernel=ploy, C={2.0}, gamma={'scale'}')
print(f'RBF SVC Train Data Score: {rbf_svc.score(x_train, y_train)}')
print(f'RBF SVC Test Data Score: {rbf_svc.score(x_test, y_test)}')

kernel=ploy, C=2.0, gamma=scale
RBF SVC 학습 데이터 평가: 0.9154929577464789
RBF SVC 평가 데이터 평가: 0.9370629370629371


## 데이터 전처리

- SVM은 입력 데이터가 정규화 되어야 좋은 성능을 보임
- 주로 모든 특성 값을 [0, 1] 범위로 맞추는 방법을 사용
- scikit-learn의 StandardScaler 또는 MinMaxScaler를 사용해 정규화

In [16]:
# 예 load_breast_cancer 데이터를 StandardScaler 를 이용해 정규화 하고 학습시켜보자
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=123)

model = SVC()

# - Score 는?

#그냥 학습
model.fit(x_train, y_train)

print(f'Train Data Score: {model.score(x_train, y_train)}')
print(f'Test Data Score: {model.score(x_test, y_test)}')

Train Data Score: 0.9014084507042254
Test Data Score: 0.9230769230769231


In [18]:
#Standarescaler로 학습
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_sc = scaler.fit_transform(x_train) # numpy 형태로 반환
x_test_sc = scaler.fit_transform(x_test)

model.fit(x_train_sc, y_train)
print(f'Train Data Score: {model.score(x_train_sc, y_train)}')
print(f'Test Data Score: {model.score(x_test_sc, y_test)}')
#훨씬 점수가 좋아졌다

Train Data Score: 0.9835680751173709
Test Data Score: 0.986013986013986


In [21]:
#MinmaxScaler로 학습
from sklearn.preprocessing import MinMaxScaler
Mnscaler = MinMaxScaler()

x_train_mn = Mnscaler.fit_transform(x_train) # numpy 형태로 반환
x_test_mn = Mnscaler.fit_transform(x_test)

model.fit(x_train_mn, y_train)
print(f'Train Data Score: {model.score(x_train_mn, y_train)}')
print(f'Test Data Score: {model.score(x_test_mn, y_test)}')

Train Data Score: 0.9812206572769953
Test Data Score: 0.9300699300699301
